In [52]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [53]:
import pickle
import pandas as pd

In [54]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/home/user/Documents/mlops-camp/mlops-zoomcamp/venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/user/Documents/mlops-camp/mlops-zoomcamp/venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [55]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [56]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [57]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

## Q1. Mean predicted duration for Feb-21 dataset?

In [58]:
import numpy as np
print(f"Mean predicted duration is {np.mean(y_pred):.2f}")

Mean predicted duration is 16.19


In [59]:
df['ride_id'] = f'2021/02_' + df.index.astype('str')

In [60]:
df['predictions'] = y_pred
df_result = df[['ride_id','predictions']].copy()

In [61]:
df_result.to_parquet(
    'df_result.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

## Q2. Size of output file

In [62]:
!du -sh df_result.parquet

19M	df_result.parquet


## Q3. Creating the scoring script

jupyter nbconvert --to script starter.ipynb

## Q4. What's the first hash for the Scikit-Learn dependency?

08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b

## Q5. Parameterize the script

What's the mean predicted duration for March 2021?

16.29

## Q6. Docker container

What's the mean predicted duration for April 2021?

16.55116